### Cleaning the dataset


In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('dirty_cafe_sales.csv')

In [4]:
df.shape

(10000, 8)

In [5]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [7]:
##Standardizing Column Names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.columns

Index(['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent',
       'payment_method', 'location', 'transaction_date'],
      dtype='object')

In [9]:
##checking for duplicates
df.duplicated().sum()
## no duplicates found
## If there were duplicates, we could remove them using:
##  df = df.drop_duplicates(inplace=True)

np.int64(0)

In [10]:
df.isnull().sum()

transaction_id         0
item                 333
quantity             138
price_per_unit       179
total_spent          173
payment_method      2579
location            3265
transaction_date     159
dtype: int64

In [19]:
df.dtypes

transaction_id      object
item                object
quantity            object
price_per_unit      object
total_spent         object
payment_method      object
location            object
transaction_date    object
dtype: object

In [21]:
## Checking unique values of columns
df['payment_method'].unique()

array(['Credit Card', 'Cash', 'UNKNOWN', 'Digital Wallet', 'ERROR', nan],
      dtype=object)

In [22]:
df['location'].unique()

array(['Takeaway', 'In-store', 'UNKNOWN', nan, 'ERROR'], dtype=object)

### Usual Best Practice
Convert to the correct data type first (with errors='coerce' to turn invalid entries into NaN).

Handle missing values next (fill or drop them as per logic).

In [23]:
# Convert Quantity
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')

# Convert Price Per Unit
df['price_per_unit'] = pd.to_numeric(df['price_per_unit'], errors='coerce')

# Convert Total Spent (replace 'ERROR' with NaN first)
df['total_spent'] = pd.to_numeric(df['total_spent'], errors='coerce')


In [24]:
# Example: fill with median or 0
df['quantity'] = df['quantity'].fillna(df['quantity'].median())
df['price_per_unit'] = df['price_per_unit'].fillna(df['price_per_unit'].median())
df['total_spent'] = df['total_spent'].fillna(df['total_spent'].median())


In [25]:
## For Categorical Columns
# Replace 'UNKNOWN' or similar placeholders with NaN
df.replace(['UNKNOWN', 'N/A', 'NA', ''], np.nan, inplace=True)

# Fill missing Payment Method & Location with 'Unknown'
df['payment_method'] = df['payment_method'].fillna('Unknown')
df['location'] = df['location'].fillna('Unknown')


In [27]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
df['transaction_date'].isnull().sum()  # Check for any conversion errors


np.int64(460)

In [28]:
df['total_spent'] = df['quantity'] * df['price_per_unit']
df['total_spent'].isnull().sum()  # Check for any NaN values in total_spent after calculation

np.int64(0)

In [29]:
df.head()

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [30]:
df.isnull().sum()  # Final check for any remaining missing values

transaction_id        0
item                677
quantity              0
price_per_unit        0
total_spent           0
payment_method        0
location              0
transaction_date    460
dtype: int64

In [32]:
## If you must keep all rows
df['item'] = df['item'].fillna('Unknown')
##missing date can be filled by  last known date before the missing one.,first known date after the missing one.,Fills missing values with a date in between surrounding dates(works only if sorted by date).
df['transaction_date'] = df['transaction_date'].fillna(method='ffill')
##filled with the last known value in the column before the missing one.
## if missing values are useless, drop them
##df.dropna(subset=['item', 'transaction_date'], inplace=True)


C:\Users\mhetr\AppData\Local\Temp\ipykernel_5156\11275983.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['transaction_date'] = df['transaction_date'].fillna(method='ffill')


In [33]:
df.isnull().sum()  # Final check for any remaining missing values

transaction_id      0
item                0
quantity            0
price_per_unit      0
total_spent         0
payment_method      0
location            0
transaction_date    0
dtype: int64

In [35]:
df.head()


,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [36]:
df.to_csv('cleaned_cafe_sales_data.csv', index=False)
